# LangChain 실습

## Data Connection

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install langchain
# !pip install openai
# !pip install pypdf
# !pip install tiktoken
# !pip install faiss-cpu
# !pip install sentence-transformers
# !pip install tf-keras

In [8]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("The_Adventures_of_Tom_Sawyer.pdf")
document = loader.load()
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he \ndidn’t like working. He liked playing and having \nadventures. One Friday, he didn’t go to school—he went \nto the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you \ndidn’t go to school today. Tomorrow you’re going to work \nfor me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because \nyou’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good pai

In [6]:
print(type(document), len(document))
print(type(document[0]), len(document[0].page_content))

<class 'list'> 35
<class 'langchain_core.documents.base.Document'> 0


In [9]:
from langchain_classic.vectorstores import FAISS
from langchain_classic.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings() #임베딩 처리
db = FAISS.from_documents(document, embeddings)

In [11]:
text = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
text_embedding = embeddings.embed_query(text)
print(text_embedding[:50])

[-0.0028138665231909497, -0.020238374776143958, -0.012738939981790699, -0.016539159741274387, -0.02259930910767389, 0.028709960370719174, -0.02787668932061184, 0.0063410627904474165, -0.012025609306402632, 0.009797242220737239, -0.015478632950228687, 0.014367605193859757, 0.0067166661065529805, -0.03338132341184305, 0.0005783971135446387, 0.02805344316490442, 0.022271049573443964, -0.00504696868553582, 0.024139597397596656, -0.027093920167150287, -0.0003393052525143871, 0.0038601892483514063, 0.01402672158245221, -0.02090751699045888, -0.0051574398382186835, 0.018306701909490415, 0.02432897625038941, -0.02041513048308164, -0.004163196670105254, -0.01867283646922085, 0.0012435932560036851, -0.0012057172991806245, -0.03547712697693411, -0.0050722191681974345, 0.001227811646132516, -0.017713313471466714, 0.01108502382139875, 0.0072658670148260844, 0.009576298984048959, 0.003251017919888031, -0.0006107494545258967, 0.003241548930682266, 0.0054288840402301934, -0.0077140655267914525, 0.0155

In [12]:
from langchain_classic.chat_models import ChatOpenAI
from langchain_classic.chains import RetrievalQA

retriever = db.as_retriever()
llm = ChatOpenAI(temperature=0, model_name='gpt-4.1-mini') # 창의성 0으로 설정
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "마을 무덤에 있던 남자를 죽인 사람은 누구니?"
result = qa({"query": query})
print(result['result'])

C:\Users\User\AppData\Local\Temp\ipykernel_1444\3104188161.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name='gpt-4.1-mini') # 창의성 0으로 설정
C:\Users\User\AppData\Local\Temp\ipykernel_1444\3104188161.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa({"query": query})


마을 무덤에 있던 남자를 죽인 사람은 인준 조(Injun Joe)입니다.
